# 利用transformers库封装model

通过上一个文件，我们已经知道了minimind-dense的torch模型了<br>
现在我们用transformers库来封装model，方便后续的上传和推理过程<br>
要素：
- tokenizer
- embedding
- minimind-block
    - RoPE
    - RMSNorm
    - GQA
    - FFN
- lm_head
- de-tokenizer

[关于transformers库](https://cloud.tencent.com/developer/article/2367010)

In [1]:
from transformers import PretrainedConfig

class MinimindConfig(PretrainedConfig):
    model_type = "minimind"
    def __init__(self,
                # tokenizer相关 
                 vocab_size = 6400,
                 model_max_length=32768,
                 eos_token_id = 1,
                 bos_token_id = 2,
                # attn 参数
                max_seqlen=512,
                embed_dim=512,
                head_num=8,
                kv_head_num=4,
                attn_dropout=0.1,
                attn_res_dropout=0.1,
                Flash=False,
                # ffn 参数,
                ffn_dim=2048,
                act_fn = "silu",
                ffn_dropout=0.1,
                # norm 参数
                rmsnorm_eps=1e-6,
                # block参数,
                block_num=8,
                ## MOE 参数
                # 其他参数
                **kwargs
                ):
        super(MinimindConfig,self).__init__(**kwargs)
        # tokenizer相关
        self.vocab_size = vocab_size
        self.model_max_length = model_max_length
        self.eos_token_id = eos_token_id
        self.bos_token_id = bos_token_id

        # 结构参数
        self.block_num = block_num
        self.embed_dim = embed_dim
        self.head_num = head_num
        self.head_dim = embed_dim // head_num
        self.kv_head_num = kv_head_num
        self.ffn_dim = ffn_dim
        self.attn_res_dropout = attn_res_dropout
        self.attn_dropout = attn_dropout
        self.ffn_dropout = ffn_dropout
        self.Flash = Flash
        self.max_seqlen = max_seqlen
        self.norm_eps = rmsnorm_eps
        self.act_fn = act_fn
        ## MOE 参数


/data/zyp/miniconda3/envs/minimind/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1.tokenizer
使用我们之前预训练数据集训练好的tokenizer

[关于tokenizer参数的相关设置](https://zhuanlan.zhihu.com/p/341994096) <br>
[关于left_padding和right_padding的讨论](https://zhuanlan.zhihu.com/p/646852375)<br>
[如何改进增强长文本处理能力](https://zhuanlan.zhihu.com/p/638976034)


In [2]:
from transformers import AutoTokenizer
tokenizer_path= "./"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
print(tokenizer)
print(tokenizer.vocab_size)
print(tokenizer.special_tokens_map)
print(tokenizer.all_special_tokens)

# 写一个虚拟的小的数据集，只有两条数据的集
data=[
    {'text':'<|im_start|>鉴别一组中文文章的风格和特点，例如官方、口语、文言等。需要提供样例文章才能准确鉴别不同的风格和特点。<|im_end|> <|im_start|>好的，现在帮我查一下今天的天气怎么样?今天的天气依据地区而异。请问你需要我帮你查询哪个地区的天气呢？<|im_end|> <|im_start|>打开闹钟功能，定一个明天早上七点的闹钟。好的，我已经帮您打开闹钟功能，闹钟将在明天早上七点准时响起。<|im_end|> <|im_start|>为以下场景写一句话描述：一个孤独的老人坐在公园长椅上看着远处。一位孤独的老人坐在公园长椅上凝视远方。<|im_end|> <|im_start|>非常感谢你的回答。请告诉我，这些数据是关于什么主题的？这些数据是关于不同年龄段的男女人口比例分布的。<|im_end|> <|im_start|>帮我想一个有趣的标题。这个挺有趣的："如何成为一名成功的魔术师" 调皮的标题往往会吸引读者的注意力。<|im_end|> <|im_start|>回答一个问题，地球的半径是多少？地球的平均半径约为6371公里，这是地球自赤道到两极的距离的平均值。<|im_end|> <|im_start|>识别文本中的语气，并将其分类为喜悦、悲伤、惊异等.文本：“今天是我的生日！”这个文本的语气是喜悦。<|im_end|>'
},
    {'text':'<|im_start|>根据输入的内容，编写一个类别标签。这是一篇介绍如何阅读心电图的文章类别标签: 医学/心电图阅读指南<|im_end|> <|im_start|>帮我搜索一下最近的天气情况。当然，我可以帮您搜索最新的天气情况。请问您需要查询哪个城市的天气情况呢？<|im_end|> <|im_start|>帮我讲一个令人开心的笑话。好的，我帮您讲一个关于细菌的笑话。为什么细菌不会上网？因为连接总是断开了！<|im_end|> <|im_start|>现在给我生成一首关于大海的五言诗。碧波万顷月满天，海天相接处天地间。波涛滚滚江山美，海鸟翱翔日月闲。<|im_end|> <|im_start|>谢谢你，这篇文章很有用。不客气，我很高兴能够为您提供帮助。如果您还有其他问题或需求，随时可以对我说。<|im_end|> <|im_start|>你好，我想下载一个视频编辑软件，你有什么推荐吗？您好！当然，有很多选择。您想要免费软件还是愿意付费？<|im_end|> <|im_start|>为什么我的程序不输出正确结果？可能是代码逻辑有误，或者输入数据有误，需要仔细调试代码逻辑和输入数据。<|im_end|> <|im_start|>谢谢你的回答。现在我想知道这场比赛的具体时间和地点。这场比赛的时间是北京时间10月4日，地点是上海。<|im_end|>'
}
]
for i in range(2):
    print(data[i]['text'])

# 接下来将该data的内容利用tokenizer编码
input_texts = [item['text'] for item in data]
#填充1 固定填充
input_ids1 = tokenizer(input_texts, padding='max_length', truncation=True, max_length=512,return_tensors='pt')
#填充2 动态填充
input_ids2 = tokenizer(input_texts, padding=True, truncation=True, max_length=512,return_tensors='pt')
print(input_ids1['input_ids'].shape)
print(input_ids2['input_ids'].shape)
print(input_ids1)
print(input_ids2)
# 这样我们就获取到了tokenizer编码后的数据
#现在我们深拷贝input_ids1
import copy
input_ids = copy.deepcopy(input_ids1['input_ids'])
# 后续我们采用这个input_ids进行模型训练

PreTrainedTokenizerFast(name_or_path='./', vocab_size=6400, model_max_length=32768, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|im_start|>', 'eos_token': '<|im_end|>', 'unk_token': '<|endoftext|>', 'pad_token': '<pad>', 'additional_special_tokens': ['<pad>', '<mask>', '<s>', '</s>', '<unk>', '<UNK>', '<EOS>', '<zzy>', '<|s1|>', '<|s2|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<endoftext>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedTo

## 2.Embedding
对应参数 vocab_size,embed_dim<br>
这里是 vocab_size=6400 embed_dim=512

In [ ]:
# 测试版
import torch 
from torch import nn
class Embed(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(Embed, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)

    def forward(self, input_ids):
        return self.embedding(input_ids)

# 测试
embed_dim=512
vocab_size=6400
embed_model = Embed(vocab_size, embed_dim)
output = embed_model(input_ids)
print(output.shape)  # 输出形状应为 (batch_size, sequence_length, embed
print(output)

#待会儿直接用就行

In [3]:
import torch 
from torch import nn
class Embed(nn.Module):
    def __init__(self, config:MinimindConfig):
        super(Embed, self).__init__()
        vocab_size = config.vocab_size
        embed_dim = config.embed_dim
        self.embedding = nn.Embedding(vocab_size, embed_dim)

    def forward(self, input_ids):
        return self.embedding(input_ids)

# 测试
embed_dim=512
vocab_size=6400
config1=MinimindConfig(vocab_size=vocab_size, embed_dim=embed_dim)
embed_model = Embed(config1)
output = embed_model(input_ids)
print(output.shape)  # 输出形状应为 (batch_size, sequence_length, embed
print(output)

torch.Size([2, 512, 512])
tensor([[[-0.4922,  0.0596, -0.8469,  ...,  1.2985,  1.7686,  0.4626],
         [-0.9253, -1.5448, -1.4958,  ..., -0.3727,  0.2492,  1.6202],
         [-0.3709, -1.0415,  2.3231,  ...,  0.1239,  0.1778, -0.2546],
         ...,
         [-0.9112, -0.1993,  0.8362,  ..., -0.1183, -1.1993,  0.7295],
         [-0.9112, -0.1993,  0.8362,  ..., -0.1183, -1.1993,  0.7295],
         [-0.9112, -0.1993,  0.8362,  ..., -0.1183, -1.1993,  0.7295]],

        [[-0.4922,  0.0596, -0.8469,  ...,  1.2985,  1.7686,  0.4626],
         [-0.5598,  1.0516,  1.0343,  ..., -2.1943, -1.5983,  0.3416],
         [ 2.0491, -0.5918, -0.9708,  ...,  0.3106, -0.7558, -0.4342],
         ...,
         [-0.9112, -0.1993,  0.8362,  ..., -0.1183, -1.1993,  0.7295],
         [-0.9112, -0.1993,  0.8362,  ..., -0.1183, -1.1993,  0.7295],
         [-0.9112, -0.1993,  0.8362,  ..., -0.1183, -1.1993,  0.7295]]],
       grad_fn=<EmbeddingBackward0>)


## 3.RMSNorm
计算公式为
$$ a_i=\frac{a_i}{RMS(a)+\epsilon} * \gamma \quad where \quad RMS(a) = \sqrt{\frac{1}{n}\sum^n_{i=1}a^2_i} $$

In [ ]:
class RMSNorm(nn.Module):
    def __init__(self,embed_dim,eps=1e-6):
        super(RMSNorm,self).__init__()
        self.embed_dim=embed_dim
        self.eps=eps
        self.gamma = nn.Parameter(torch.ones(embed_dim))
    
    def forward(self,x):
        return x*self.gamma*torch.rsqrt(x.pow(2).mean(dim=-1,keepdim=True)+self.eps)

# 测试RMSNorm
rmsnorm_model = RMSNorm(embed_dim)
output_rmsnorm = rmsnorm_model(output)
print(output_rmsnorm)  # 输出形状应为 (batch_size, sequence

In [4]:
class RMSNorm(nn.Module):
    def __init__(self,config:MinimindConfig):
        super(RMSNorm,self).__init__()
        self.embed_dim = config.embed_dim
        self.eps = config.norm_eps
        self.gamma = nn.Parameter(torch.ones(embed_dim))
    
    def forward(self,x):
        return x*self.gamma*torch.rsqrt(x.pow(2).mean(dim=-1,keepdim=True)+self.eps)

# 测试RMSNorm
rmsnorm_model = RMSNorm(config1)
output_rmsnorm = rmsnorm_model(output)
print(output_rmsnorm)  # 输出形状应为 (batch_size, sequence_length

tensor([[[-0.5054,  0.0612, -0.8696,  ...,  1.3332,  1.8159,  0.4750],
         [-0.9202, -1.5363, -1.4876,  ..., -0.3706,  0.2478,  1.6113],
         [-0.3675, -1.0320,  2.3018,  ...,  0.1228,  0.1762, -0.2522],
         ...,
         [-0.9005, -0.1969,  0.8263,  ..., -0.1169, -1.1852,  0.7208],
         [-0.9005, -0.1969,  0.8263,  ..., -0.1169, -1.1852,  0.7208],
         [-0.9005, -0.1969,  0.8263,  ..., -0.1169, -1.1852,  0.7208]],

        [[-0.5054,  0.0612, -0.8696,  ...,  1.3332,  1.8159,  0.4750],
         [-0.5673,  1.0656,  1.0480,  ..., -2.2235, -1.6196,  0.3461],
         [ 2.0147, -0.5819, -0.9545,  ...,  0.3054, -0.7431, -0.4269],
         ...,
         [-0.9005, -0.1969,  0.8263,  ..., -0.1169, -1.1852,  0.7208],
         [-0.9005, -0.1969,  0.8263,  ..., -0.1169, -1.1852,  0.7208],
         [-0.9005, -0.1969,  0.8263,  ..., -0.1169, -1.1852,  0.7208]]],
       grad_fn=<MulBackward0>)


## 4.RoPE
主要两步骤
- 获取$ m\theta $ ,计算好precompute_pos_cis
- 将pos_cis应用

主要公式为：
$$
\begin{align}
f_q(\boldsymbol{x}_m, m) &= (\boldsymbol{W}_q \boldsymbol{x}_m) e^{im\theta} \\
f_k(\boldsymbol{x}_n, n) &= (\boldsymbol{W}_k \boldsymbol{x}_n) e^{in\theta} \\
g(\boldsymbol{x}_m, \boldsymbol{x}_n, m - n) &= \text{Re}\left[ (\boldsymbol{W}_q \boldsymbol{x}_m)^* (\boldsymbol{W}_k \boldsymbol{x}_n) e^{i(n - m)\theta} \right]
\end{align}
$$

$\Theta=\left\{\theta_i=10000^{-2(i-1)/d},i\in[1,2,\ldots,d/2]\right\}$

In [5]:
def precompute_pos_cis(embed_dim=512,max_seqlen=512,theta=1e5):
    freqs= 1/theta**torch.arange(0,embed_dim,2)[:embed_dim//2].float()
    m=torch.arange(max_seqlen,device=freqs.device)
    freqs= torch.outer(m,freqs).float() #获取了mtheta
    pos_cis = torch.polar(torch.ones_like(freqs),freqs) #将mtheta化为极坐标模式
    return pos_cis

def apply_rotary(xq,xk,pos_cis):
    xq_=torch.view_as_complex(xq.float().reshape(*xq.shape[:-1],-1,2))
    xk_=torch.view_as_complex(xk.float().reshape(*xk.shape[:-1],-1,2))
    #输入的pos_cis一般比xq,xk都要大，需要把pos_cis的形状和xq对齐
    #xq一般都是(bs,seqlen,head,head_dim)
    def unite_shape(pos_cis,  x):
        ndim = x.ndim
        assert 0 <= 1 < ndim
        assert pos_cis.shape == (x.shape[1],  x.shape[-1]), f"pos_cis shape {pos_cis.shape} does not match x shape {x.shape}"
        shape = [d if i == 1 or i == ndim - 1 else 1 for i,  d in enumerate(x.shape)]
        return pos_cis.view(*shape)
    pos_cis = unite_shape(pos_cis, xq_)
    xq_ = torch.view_as_real(xq_ * pos_cis).flatten(3)
    xk_ = torch.view_as_real(xk_ * pos_cis).flatten(3)
    return xq_, xk_

#测试一下
pos_cis = precompute_pos_cis(embed_dim=embed_dim, max_seqlen=512)
print(pos_cis)
print(output_rmsnorm.shape)  # 输出形状应为 (batch_size, sequence_length, embed_dim)
xq, xk = apply_rotary(output_rmsnorm, output_rmsnorm, pos_cis)
print(xq.shape)  # 输出形状应为 (batch_size,
print(xk.shape)  # 输出形状应为 (batch_size, sequence_length, embed_dim)
# 测试一下apply_rotary的效果



tensor([[ 1.0000+0.0000e+00j,  1.0000+0.0000e+00j,  1.0000+0.0000e+00j,
          ...,  1.0000+0.0000e+00j,  1.0000+0.0000e+00j,
          1.0000+0.0000e+00j],
        [ 0.5403+8.4147e-01j,  1.0000+1.0000e-10j,  1.0000+1.0000e-20j,
          ...,  1.0000+0.0000e+00j,  1.0000+0.0000e+00j,
          1.0000+0.0000e+00j],
        [-0.4161+9.0930e-01j,  1.0000+2.0000e-10j,  1.0000+2.0000e-20j,
          ...,  1.0000+0.0000e+00j,  1.0000+0.0000e+00j,
          1.0000+0.0000e+00j],
        ...,
        [ 0.9981+6.1950e-02j,  1.0000+5.0900e-08j,  1.0000+5.0900e-18j,
          ...,  1.0000+0.0000e+00j,  1.0000+0.0000e+00j,
          1.0000+0.0000e+00j],
        [ 0.4871+8.7333e-01j,  1.0000+5.1000e-08j,  1.0000+5.1000e-18j,
          ...,  1.0000+0.0000e+00j,  1.0000+0.0000e+00j,
          1.0000+0.0000e+00j],
        [-0.4717+8.8177e-01j,  1.0000+5.1100e-08j,  1.0000+5.1100e-18j,
          ...,  1.0000+0.0000e+00j,  1.0000+0.0000e+00j,
          1.0000+0.0000e+00j]])
torch.Size([2, 512, 512])


## 5.GQA
![LLM-结构](../images/LLM-structure.png)

In [6]:
# repeat_kv是必须要用到的,对齐GQA里的KV与Q的形状
def repeat_kv(x,rep_num):
    if rep_num == 1:
        return x
    bs,seqlen,head,head_dim=x.shape
    return x[:,:,:,None,:].expand(bs,seqlen,head,rep_num,head_dim).reshape(bs,seqlen,head*rep_num,head_dim)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
class GroupQueryAttention(nn.Module):
    def __init__(self,embed_dim,head_num,kv_head_num,attn_dropout=0.1,attn_res_dropout=0.1,Flash=False,max_seqlen=512):
        super(GroupQueryAttention,self).__init__()
        #基本属性
        self.embed_dim = embed_dim
        self.head_num = head_num
        self.kv_head_num = kv_head_num
        self.head_dim = embed_dim // head_num
        assert embed_dim % head_num == 0, "embed_dim must be divisible by head_num"
        self.rep_num = head_num // kv_head_num
        assert head_num % kv_head_num == 0, "kv_head_num must be divisible by head_num"
        self.Flash = hasattr(torch.nn.functional,'scaled_dot_product_attention') and Flash
        #网络层
        self.q_proj = nn.Linear(embed_dim,self.head_num * self.head_dim)
        self.k_proj = nn.Linear(embed_dim,self.kv_head_num * self.head_dim)
        self.v_proj = nn.Linear(embed_dim,self.kv_head_num * self.head_dim)
        self.o_proj = nn.Linear(self.head_num*self.head_dim,self.embed_dim)
        self.attn_dropout = nn.Dropout(attn_dropout)
        self.res_dropout = nn.Dropout(attn_res_dropout)
        #临时
        mask = torch.full((1,1, max_seqlen, max_seqlen), float('-1e9'))
        mask = torch.tril(mask, diagonal=0)
        self.register_buffer('mask', mask)
    def forward(self,x,
                pos_cis=None,
                past_key_value=None,
                use_cache=False):
        bs,seqlen,embed_dim = x.shape
        xq = self.q_proj(x)
        xk = self.k_proj(x)
        xv = self.v_proj(x)

        xq = xq.view(bs, seqlen, self.head_num, self.head_dim)
        xk = xk.view(bs, seqlen, self.kv_head_num, self.head_dim)
        xv = xv.view(bs, seqlen, self.kv_head_num, self.head_dim)
        if pos_cis is None:
            pos_cis = precompute_pos_cis(embed_dim=self.head_dim, max_seqlen=seqlen)
        xq, xk = apply_rotary(xq, xk, pos_cis)
        if past_key_value is not None:
            xk = torch.cat([past_key_value[0], xk], dim=1)
            xv = torch.cat([past_key_value[1], xv], dim=1)
        past_kv = (xk, xv) if use_cache else None
        xq = xq.transpose(1,2)
        xk = repeat_kv(xk, self.rep_num).transpose(1,2)
        xv = repeat_kv(xv, self.rep_num).transpose(1,2)
        if self.Flash:
            attn_output = F.scaled_dot_product_attention(
                xq, xk, xv, attn_mask=None, dropout_p=self.dropout,is_causal=True)
        else:
            scores = torch.matmul(xq, xk.transpose(-2, -1)) / (math.sqrt(self.head_dim))
            scores+= self.mask[:, :, :seqlen, :seqlen]
            attn_weights = F.softmax(scores, dim=-1)
            attn_weights = self.attn_dropout(attn_weights)
            attn_output = torch.matmul(attn_weights, xv)
        attn_output = attn_output.transpose(1, 2).reshape(bs, seqlen, -1)
        attn_output = self.o_proj(attn_output)
        attn_output = self.res_dropout(attn_output)
        return attn_output, past_kv

# 测试GroupQueryAttention
embed_dim = 512
gqa_model = GroupQueryAttention(embed_dim=embed_dim, head_num=8, kv_head_num=4, attn_dropout=0.1, Flash=False, max_seqlen=512)
output_gqa, past_kv = gqa_model(output_rmsnorm, pos_cis=None, use_cache=False)
print(output_gqa.shape)  # 输出形状应为 (batch_size, sequence_length, embed_dim)       

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
class GroupQueryAttention(nn.Module):
    def __init__(self,config:MinimindConfig):
        super(GroupQueryAttention,self).__init__()
        #基本属性
        self.embed_dim = config.embed_dim
        self.head_num = config.head_num
        self.kv_head_num = config.kv_head_num
        self.head_dim = embed_dim // self.head_num
        assert embed_dim % self.head_num == 0, "embed_dim must be divisible by head_num"
        self.rep_num = self.head_num // self.kv_head_num
        assert self.head_num % self.kv_head_num == 0, "kv_head_num must be divisible by head_num"
        self.Flash = hasattr(torch.nn.functional,'scaled_dot_product_attention') and config.Flash
        #网络层
        self.q_proj = nn.Linear(embed_dim,self.head_num * self.head_dim)
        self.k_proj = nn.Linear(embed_dim,self.kv_head_num * self.head_dim)
        self.v_proj = nn.Linear(embed_dim,self.kv_head_num * self.head_dim)
        self.o_proj = nn.Linear(self.head_num*self.head_dim,self.embed_dim)
        self.attn_dropout = nn.Dropout(config.attn_dropout)
        self.res_dropout = nn.Dropout(config.attn_res_dropout)
        self.max_seqlen = config.max_seqlen
        #临时
        mask = torch.full((1,1, self.max_seqlen, self.max_seqlen), float('-1e9'))
        mask = torch.tril(mask, diagonal=0)
        self.register_buffer('mask', mask)
    def forward(self,x,
                pos_cis=None,
                past_key_value=None,
                use_cache=False):
        bs,seqlen,embed_dim = x.shape
        xq = self.q_proj(x)
        xk = self.k_proj(x)
        xv = self.v_proj(x)

        xq = xq.view(bs, seqlen, self.head_num, self.head_dim)
        xk = xk.view(bs, seqlen, self.kv_head_num, self.head_dim)
        xv = xv.view(bs, seqlen, self.kv_head_num, self.head_dim)
        if pos_cis is None:
            pos_cis = precompute_pos_cis(embed_dim=self.head_dim, max_seqlen=seqlen)
        xq, xk = apply_rotary(xq, xk, pos_cis)
        if past_key_value is not None:
            xk = torch.cat([past_key_value[0], xk], dim=1)
            xv = torch.cat([past_key_value[1], xv], dim=1)
        past_kv = (xk, xv) if use_cache else None
        xq = xq.transpose(1,2)
        xk = repeat_kv(xk, self.rep_num).transpose(1,2)
        xv = repeat_kv(xv, self.rep_num).transpose(1,2)
        if self.Flash:
            attn_output = F.scaled_dot_product_attention(
                xq, xk, xv, attn_mask=None, dropout_p=self.dropout,is_causal=True)
        else:
            scores = torch.matmul(xq, xk.transpose(-2, -1)) / (math.sqrt(self.head_dim))
            scores+= self.mask[:, :, :seqlen, :seqlen]
            attn_weights = F.softmax(scores, dim=-1)
            attn_weights = self.attn_dropout(attn_weights)
            attn_output = torch.matmul(attn_weights, xv)
        attn_output = attn_output.transpose(1, 2).reshape(bs, seqlen, -1)
        attn_output = self.o_proj(attn_output)
        attn_output = self.res_dropout(attn_output)
        return attn_output, past_kv

# 测试GroupQueryAttention
config2 = MinimindConfig()
gqa_model = GroupQueryAttention(config2)
output_gqa, past_kv = gqa_model(output_rmsnorm, pos_cis=None, use_cache=False)
print(output_gqa.shape)  # 输出形状应为 (

torch.Size([2, 512, 512])


## 6.FFN

In [ ]:
from transformers.activations import ACT2FN
from torch import nn
class FeedForward(nn.Module):
    def __init__(self,embed_dim,ffn_dim,ffn_dropout=0.1,act_fn ='silu'):
        super(FeedForward,self).__init__()
        self.gate = nn.Linear(embed_dim, ffn_dim)
        self.up_proj = nn.Linear(embed_dim,ffn_dim)
        self.down_proj = nn.Linear(ffn_dim, embed_dim)
        self.dropout = nn.Dropout(ffn_dropout)
        self.act_fn = ACT2FN[act_fn]
    def forward(self,x):
        return self.down_proj(self.dropout(self.act_fn(self.gate(x)) * self.up_proj(x)))

# 测试FeedForward
ffn_dim = 2048
embed_dim = 512
ffn_model = FeedForward(embed_dim=embed_dim, ffn_dim=ffn_dim, ffn_dropout=0.1)
output_ffn = ffn_model(output_gqa)
print(output_ffn.shape)  # 输出形状应为 (batch_size, sequence_length, embed_dim)


In [8]:
from transformers.activations import ACT2FN
from torch import nn
class FeedForward(nn.Module):
    def __init__(self,config:MinimindConfig):
        super(FeedForward,self).__init__()
        self.embed_dim = config.embed_dim
        self.ffn_dim = config.ffn_dim
        self.ffn_dropout = config.ffn_dropout
        self.act_fn = config.act_fn
        self.gate = nn.Linear(self.embed_dim, self.ffn_dim)
        self.up_proj = nn.Linear(self.embed_dim,self.ffn_dim)
        self.down_proj = nn.Linear(self.ffn_dim, self.embed_dim)
        self.dropout = nn.Dropout(self.ffn_dropout)
        self.act_fn = ACT2FN[self.act_fn]
    def forward(self,x):
        return self.down_proj(self.dropout(self.act_fn(self.gate(x)) * self.up_proj(x)))

ffn2 = FeedForward(config2)
output_ffn = ffn2(output_gqa)
print(output_ffn.shape)  # 输出形状应为 (batch_size, sequence_length

torch.Size([2, 512, 512])


# 7.Minimind_Block

In [ ]:
class Minimind_Block(nn.Module):
    def __init__(self,layer_id,embed_dim,head_num,kv_head_num,ffn_dim,attn_dropout=0.1,attn_res_dropout=0.1,ffn_dropout=0.1,Flash=False,max_seqlen=512,act_fn="silu"):
        super(Minimind_Block,self).__init__()
        self.layer_id = layer_id
        self.embed_dim = embed_dim
        self.head_num = head_num
        self.kv_head_num = kv_head_num
        self.ffn_dim = ffn_dim
        self.attn_dropout = attn_dropout
        self.res_attn_dropout = attn_res_dropout
        self.ffn_dropout = ffn_dropout
        self.Flash = Flash
        self.max_seqlen = max_seqlen
        self.act_fn = act_fn
        
        self.attention = GroupQueryAttention(embed_dim, head_num, kv_head_num, attn_dropout,attn_res_dropout, Flash, max_seqlen)
        self.rmsnorm1 = RMSNorm(embed_dim)
        self.ffn = FeedForward(embed_dim, ffn_dim, ffn_dropout,act_fn)
        self.rmsnorm2 = RMSNorm(embed_dim)
    def forward(self, x, pos_cis=None, past_key_value=None, use_cache=False):
        # norm1
        x = self.rmsnorm1(x)
        # attention
        attn_output, past_kv = self.attention(x, pos_cis=pos_cis, past_key_value=past_key_value, use_cache=use_cache)
        # residual connection
        x = x + attn_output
        # norm2
        x = self.rmsnorm2(x)
        # feed forward
        ffn_output = self.ffn(x)
        # residual connection
        x = x + ffn_output
        return x, past_kv
# 测试Minimind_Block
layer_id = 1
embed_dim = 512
head_num = 8
kv_head_num = 4
ffn_dim = 2048
minimind_block = Minimind_Block(layer_id, embed_dim, head_num, kv_head_num, ffn_dim, Flash=False, max_seqlen=512)
output_minimind, past_kv = minimind_block(output_ffn, pos_cis=None, use_cache=True)
print(output_minimind.shape)  # 输出形状应为 (batch_size, sequence_length, embed_dim)
print(past_kv[0].shape)  # 输出形状应为 (batch_size, sequence_length, kv_head_num * head_dim)


In [9]:
class Minimind_Block(nn.Module):
    def __init__(self,layer_id,config:MinimindConfig):
        super(Minimind_Block,self).__init__()
        self.layer_id = layer_id
        self.embed_dim = config.embed_dim
        self.head_num = config.head_num
        self.kv_head_num = config.kv_head_num
        self.ffn_dim = config.ffn_dim
        self.attn_dropout = config.attn_dropout
        self.res_attn_dropout = config.attn_res_dropout
        self.ffn_dropout = config.ffn_dropout
        self.Flash = config.Flash
        self.max_seqlen = config.max_seqlen
        self.act_fn = config.act_fn
        
        self.attention = GroupQueryAttention(config)
        self.rmsnorm1 = RMSNorm(config)
        self.ffn = FeedForward(config)
        self.rmsnorm2 = RMSNorm(config)
    def forward(self, x, pos_cis=None, past_key_value=None, use_cache=False):
        # norm1
        x = self.rmsnorm1(x)
        # attention
        attn_output, past_kv = self.attention(x, pos_cis=pos_cis, past_key_value=past_key_value, use_cache=use_cache)
        # residual connection
        x = x + attn_output
        # norm2
        x = self.rmsnorm2(x)
        # feed forward
        ffn_output = self.ffn(x)
        # residual connection
        x = x + ffn_output
        return x, past_kv

# 测试Minimind_Block
config3 = MinimindConfig()
layer_id = 1
minimind_block = Minimind_Block(layer_id,config3)
output_minimind, past_kv = minimind_block(output_ffn, pos_cis=None, use_cache=True)
print(output_minimind.shape)  # 输出形状应为

torch.Size([2, 512, 512])


## 8.Minimind_Dense

In [ ]:
class Minimind_Dense(nn.Module):
    def __init__(self,block_num,embed_dim,head_num,kv_head_num,ffn_dim,attn_dropout=0.1,attn_res_dropout=0.1,ffn_dropout=0.1,Flash=False,max_seqlen=512,act_fn="silu"):
        super(Minimind_Dense,self).__init__()
        self.block_num = block_num
        self.embed_dim = embed_dim
        self.head_num = head_num
        self.kv_head_num = kv_head_num
        self.ffn_dim = ffn_dim
        self.attn_dropout = attn_dropout
        self.res_attn_dropout = attn_res_dropout
        self.ffn_dropout = ffn_dropout
        self.Flash = Flash
        self.max_seqlen = max_seqlen
        self.act_fn = act_fn
        self.blocks = nn.ModuleList([
            Minimind_Block(layer_id, embed_dim, head_num, kv_head_num, ffn_dim, attn_dropout,attn_res_dropout,ffn_dropout, Flash, max_seqlen,act_fn)
            for layer_id in range(block_num)
        ])
    def forward(self, x, pos_cis=None, past_key_values=None, use_cache=False):
        if past_key_values is None:
            past_key_values = [None] * self.block_num
        for i, block in enumerate(self.blocks):
            x, past_kv = block(x, pos_cis=pos_cis, past_key_value=past_key_values[i], use_cache=use_cache)
            if use_cache:
                past_key_values[i] = past_kv
        return x, past_key_values
# 测试Minimind_Dense
block_num = 6
minimind_dense = Minimind_Dense(block_num, embed_dim, head_num, kv_head_num, ffn_dim, dropout=0.1, Flash=False, max_seqlen=512)
output_minimind_dense, past_key_values = minimind_dense(output_minimind, pos_cis=None, use_cache=True)
print(output_minimind_dense.shape)  # 输出形状应为 (batch_size, sequence_length, embed_dim)
print(len(past_key_values))  # 输出块数
print(past_key_values[0][0].shape)  # 输出形状应为 (batch_size, sequence_length, kv_head_num * head_dim


In [10]:
class Minimind_Dense(nn.Module):
    def __init__(self,config:MinimindConfig):
        super(Minimind_Dense,self).__init__()
        self.blocks = nn.ModuleList([
            Minimind_Block(layer_id, config)
            for layer_id in range(config.block_num)
        ])
    def forward(self, x, pos_cis=None, past_key_values=None, use_cache=False):
        if past_key_values is None:
            past_key_values = [None] * self.block_num
        for i, block in enumerate(self.blocks):
            x, past_kv = block(x, pos_cis=pos_cis, past_key_value=past_key_values[i], use_cache=use_cache)
            if use_cache:
                past_key_values[i] = past_kv
        return x, past_key_values

## transformers包装Minimind_Dense

![.](../images/LLM-structure.png)

In [14]:
from transformers import PreTrainedModel,GenerationMixin
from transformers.modeling_outputs import CausalLMOutputWithPast

class MinimindForCausalLM(PreTrainedModel,GenerationMixin):
    config_class = MinimindConfig
    base_model_prefix = "minimind"
    def __init__(self,params:MinimindConfig = None):
        self.params = params if params is not None else MinimindConfig()
        super(MinimindForCausalLM,self).__init__(self.params)
        self.embed = Embed(config=self.params)
        self.rmsnorm = RMSNorm(config=self.params)
        self.minimind_dense = Minimind_Dense(config=self.params)
        self.lm_head = nn.Linear(params.embed_dim, params.vocab_size)
        ## 临时属性
        pos_cis = precompute_pos_cis(embed_dim=params.head_dim, max_seqlen=params.max_seqlen)
        self.register_buffer('pos_cis', pos_cis,persistent=False)
        self.OUT = CausalLMOutputWithPast()
    
    def forward(self,input_ids=None,
                past_key_values=None,
                use_cache = False,
                **args):
        past_key_values = past_key_values if past_key_values is not None else [None] * self.params.block_num
        start_pos = args.get('start_pos', 0)
        hidden_states = self.embed(input_ids)
        pos_cis = self.pos_cis[start_pos:start_pos+hidden_states.size(1)]
        past_kvs = []
        ### 
        print("######FORWARD GENERATION######")
        print(f"input_ids shape: {input_ids.shape if input_ids is not None else 'None'}")
        for i, block in enumerate(self.minimind_dense.blocks):
            hidden_states, past_kv = block(hidden_states, pos_cis=pos_cis, past_key_value=past_key_values[i], use_cache=use_cache)
            if use_cache:
                past_kvs.append(past_kv)
            else:
                past_kvs.append(None)
            print(f"Block {i+1}/{self.params.block_num} processed, hidden_states shape: {hidden_states.shape}")
        hidden_states = self.rmsnorm(hidden_states)
        logits = self.lm_head(hidden_states)
        print("######FORWARD GENERATION END######")
        self.OUT.__setitem__('logits', logits)
        self.OUT.__setitem__('past_key_values', past_kvs)
        self.OUT.__setitem__('hidden_states', hidden_states)
        return self.OUT
    
    # 推理函数（包含top-p或者top-k等策略）
    @torch.inference_mode()
    def generate(self,  input_ids,  eos_token_id=1,  max_new_tokens=512,  temperature=0.75,  top_p=0.90, 
                stream=False,  rp=1,  use_cache=True,  pad_token_id=3,  **args):
        # 流式生成 （返回生成器, 自由控制输出）
        if stream:
            return self._stream(input_ids,  eos_token_id,  max_new_tokens,  temperature,  top_p,  rp,  use_cache,  **args)
        # 直接生成 （一步到位）
        generated = []
        for i in range(input_ids.size(0)):
            non_pad = input_ids[i][input_ids[i] != pad_token_id].unsqueeze(0)
            out = self._stream(non_pad,  eos_token_id,  max_new_tokens,  temperature,  top_p,  rp,  use_cache,  **args)
            tokens_list = [tokens[:,  -1:] for tokens in out]
            print(f'new tokens list :{tokens_list}\n')
            gen = torch.cat(tokens_list,  dim=-1) if tokens_list else non_pad
            full_sequence = torch.cat([non_pad,  gen],  dim=-1)
            generated.append(full_sequence)
        max_length = max(seq.size(1) for seq in generated)
        generated = [
            torch.cat([seq,  torch.full((1,  max_length - seq.size(1)),  pad_token_id,  dtype=seq.dtype,  device=seq.device)], dim=-1) 
            for seq in generated
        ]
        return torch.cat(generated,  dim=0)

    # 流式输出
    def _stream(self,  input_ids,  eos_token_id,  max_new_tokens,  temperature,  top_p,  rp,  use_cache,  **args):
        start,  first_seq,  past_kvs = input_ids.shape[1],  True,  None
        new_token_idx = 0 #  new token 计数器
        while input_ids.shape[1] < max_new_tokens - 1:
            print(f'gernerating new token: idx = {start + new_token_idx}')
            if first_seq or not use_cache: # 若第一次生成序列 or 无 KV Cache,  每次生成传入整个 token id 序列
                out,  first_seq = self(input_ids,  past_key_values=past_kvs,  use_cache=use_cache,  **args),  False
            else: # 若非第一次生成 and 有 KV Cache, 每次传入最后一个 token id 与 KV Cache 进行推理加速
                out = self(input_ids[:,  -1:],  past_key_values=past_kvs,  use_cache=use_cache, 
                           start_pos=input_ids.shape[1] - 1,  **args)
            logits,  past_kvs = out.logits[:,  -1,  :],  out.past_key_values # logits.shape: (batch_size,  seq_len,  vocab_size), 获取最后一位 logits
            logits[:,  list(set(input_ids.tolist()[0]))] /= rp # 对生成 token 进行惩罚, 降低后续重复生成几率
            logits /= (temperature + 1e-9) # 调整温度, 控制生成多样性
            if top_p is not None and top_p < 1.0: # top-p 采样
                sorted_logits,  sorted_indices = torch.sort(logits,  descending=True,  dim=-1)
                sorted_probs = F.softmax(sorted_logits,  dim=-1)
                cumulative_probs = torch.cumsum(sorted_probs,  dim=-1)
                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[:,  1:] = sorted_indices_to_remove[:,  :-1].clone()
                sorted_indices_to_remove[:,  0] = False
                indices_to_remove = sorted_indices_to_remove.scatter(1,  sorted_indices,  sorted_indices_to_remove)
                logits[indices_to_remove] = -float('Inf')
            input_ids_next = torch.multinomial(F.softmax(logits,  dim=-1),  num_samples=1) # 从保留的 token 中采样
            input_ids = torch.cat((input_ids,  input_ids_next),  dim=1)
            new_token_idx += 1
            yield input_ids[:,  start:]
            if input_ids_next.item() == eos_token_id:
                break

### 最终测试model


In [15]:
config_dense = MinimindConfig()
model = MinimindForCausalLM(config_dense)
print(model)
# 生成新的token
outputs2 = model.generate(input_ids=input_ids, max_new_tokens=10, use_cache=True,)
# 从最初的tokenizer解码outputs中取logits进行解码
print(outputs2.shape)  # 输出形状应为 (batch_size, sequence_length + max_new_tokens)
tokenizer.decode(outputs2[1], skip_special_tokens=False)  # 解码第一个batch的输出

MinimindForCausalLM(
  (embed): Embed(
    (embedding): Embedding(6400, 512)
  )
  (rmsnorm): RMSNorm()
  (minimind_dense): Minimind_Dense(
    (blocks): ModuleList(
      (0-7): 8 x Minimind_Block(
        (attention): GroupQueryAttention(
          (q_proj): Linear(in_features=512, out_features=512, bias=True)
          (k_proj): Linear(in_features=512, out_features=256, bias=True)
          (v_proj): Linear(in_features=512, out_features=256, bias=True)
          (o_proj): Linear(in_features=512, out_features=512, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (res_dropout): Dropout(p=0.1, inplace=False)
        )
        (rmsnorm1): RMSNorm()
        (ffn): FeedForward(
          (gate): Linear(in_features=512, out_features=2048, bias=True)
          (up_proj): Linear(in_features=512, out_features=2048, bias=True)
          (down_proj): Linear(in_features=2048, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (ac

'<|im_start|>根据输入的内容，编写一个类别标签。这是一篇介绍如何阅读心电图的文章类别标签: 医学/心电图阅读指南<|im_end|> <|im_start|>帮我搜索一下最近的天气情况。当然，我可以帮您搜索最新的天气情况。请问您需要查询哪个城市的天气情况呢？<|im_end|> <|im_start|>帮我讲一个令人开心的笑话。好的，我帮您讲一个关于细菌的笑话。为什么细菌不会上网？因为连接总是断开了！<|im_end|> <|im_start|>现在给我生成一首关于大海的五言诗。碧波万顷月满天，海天相接处天地间。波涛滚滚江山美，海鸟翱翔日月闲。<|im_end|> <|im_start|>谢谢你，这篇文章很有用。不客气，我很高兴能够为您提供帮助。如果您还有其他问题或需求，随时可以对我说。<|im_end|> <|im_start|>你好，我想下载一个视频编辑软件，你有什么推荐吗？您好！当然，有很多选择。您想要免费软件还是愿意付费？<|im_end|> <|im_start|>为什么我的程序不输出正确结果？可能是代码逻辑有误，或者输入数据有误，需要仔细调试代码逻辑和输入数据。<|im_end|> <|im_start|>谢谢你的回答。现在我想知道这场比赛的具体时间和地点。这场比赛的时间是北京时间10月4日，地点是上海。<|im_end|><|im_start|>根据输入的内容，编写一个类别标签。这是一篇介绍如何阅读心电图的文章类别标签: 医学/心电图阅读指南<|im_end|> <|im_start|>帮我搜索一下最近的天气情况。当然，我可以帮您搜索最新的天气情况。请问您需要查询哪个城市的天气情况呢？<|im_end|> <|im_start|>帮我讲一个令人开心的笑话。好的，我帮您讲一个关于细菌的笑话。为什么细菌不会上网？因为连接总是断开了！<|im_end|> <|im_start|>现在给我生成一首关于大海的五言诗。碧波万顷月满天，海天相接处天地间。波涛滚滚江山美，海鸟翱翔日月闲。<|im_end|> <|im_start|>谢谢你，这篇文章很有用。不客气，我很高兴能够为您提供帮助。如果您还有其他问题或需求，随时可以对我说。<|im_end|> <|im_start|>你好，我想下载一个视频编辑软件，你有什么推荐吗？您好！当然，有很多选择。您想要免费

In [18]:
config_dense2 = MinimindConfig()
model2 = MinimindForCausalLM(config_dense2)
input_test2=tokenizer("<|im_start|>你好", return_tensors='pt')
print(input_test2)
outputs3 = model2.generate(input_ids=input_test2['input_ids'], max_new_tokens=10, use_cache=True,)
print(outputs3.shape)  # 输出形状应为 (batch_size, sequence_length +
tokenizer.decode(outputs3[0], skip_special_tokens=False)  # 解码第一个batch的输出

{'input_ids': tensor([[   2, 2899]]), 'token_type_ids': tensor([[0, 0]]), 'attention_mask': tensor([[1, 1]])}
gernerating new token: idx = 2
######FORWARD GENERATION######
input_ids shape: torch.Size([1, 2])
Block 1/8 processed, hidden_states shape: torch.Size([1, 2, 512])
Block 2/8 processed, hidden_states shape: torch.Size([1, 2, 512])
Block 3/8 processed, hidden_states shape: torch.Size([1, 2, 512])
Block 4/8 processed, hidden_states shape: torch.Size([1, 2, 512])
Block 5/8 processed, hidden_states shape: torch.Size([1, 2, 512])
Block 6/8 processed, hidden_states shape: torch.Size([1, 2, 512])
Block 7/8 processed, hidden_states shape: torch.Size([1, 2, 512])
Block 8/8 processed, hidden_states shape: torch.Size([1, 2, 512])
######FORWARD GENERATION END######
gernerating new token: idx = 3
######FORWARD GENERATION######
input_ids shape: torch.Size([1, 1])
Block 1/8 processed, hidden_states shape: torch.Size([1, 1, 512])
Block 2/8 processed, hidden_states shape: torch.Size([1, 1, 512])

'<|im_start|>你好会在率慢的感觉可能是完成任务哥'

In [19]:
# 写一个虚拟的小的数据集，只有两条数据的集
data2=[
    {'text':'<|im_start|>鉴别一组中文文章的风格和特点<|im_end|> '
},
    {'text':'<|im_start|>根据输入的内容，编写一个类别标签。<|im_end|> <|im_start|>'
}
]
for i in range(2):
    print(data[i]['text'])

# 接下来将该data的内容利用tokenizer编码
input_texts = [item['text'] for item in data]
input_test3 = tokenizer(input_texts, padding='max_length', truncation=True, max_length=512,return_tensors='pt')
outputs4 = model2.generate(input_ids=input_test3['input_ids'], max_new_tokens=10, use_cache=True,)
print(outputs4.shape)  # 输出形状应为 (batch_size, sequence_length +
tokenizer.decode(outputs4[0], skip_special_tokens=False)  # 解码第一个batch的输出

<|im_start|>鉴别一组中文文章的风格和特点，例如官方、口语、文言等。需要提供样例文章才能准确鉴别不同的风格和特点。<|im_end|> <|im_start|>好的，现在帮我查一下今天的天气怎么样?今天的天气依据地区而异。请问你需要我帮你查询哪个地区的天气呢？<|im_end|> <|im_start|>打开闹钟功能，定一个明天早上七点的闹钟。好的，我已经帮您打开闹钟功能，闹钟将在明天早上七点准时响起。<|im_end|> <|im_start|>为以下场景写一句话描述：一个孤独的老人坐在公园长椅上看着远处。一位孤独的老人坐在公园长椅上凝视远方。<|im_end|> <|im_start|>非常感谢你的回答。请告诉我，这些数据是关于什么主题的？这些数据是关于不同年龄段的男女人口比例分布的。<|im_end|> <|im_start|>帮我想一个有趣的标题。这个挺有趣的："如何成为一名成功的魔术师" 调皮的标题往往会吸引读者的注意力。<|im_end|> <|im_start|>回答一个问题，地球的半径是多少？地球的平均半径约为6371公里，这是地球自赤道到两极的距离的平均值。<|im_end|> <|im_start|>识别文本中的语气，并将其分类为喜悦、悲伤、惊异等.文本：“今天是我的生日！”这个文本的语气是喜悦。<|im_end|>
<|im_start|>根据输入的内容，编写一个类别标签。这是一篇介绍如何阅读心电图的文章类别标签: 医学/心电图阅读指南<|im_end|> <|im_start|>帮我搜索一下最近的天气情况。当然，我可以帮您搜索最新的天气情况。请问您需要查询哪个城市的天气情况呢？<|im_end|> <|im_start|>帮我讲一个令人开心的笑话。好的，我帮您讲一个关于细菌的笑话。为什么细菌不会上网？因为连接总是断开了！<|im_end|> <|im_start|>现在给我生成一首关于大海的五言诗。碧波万顷月满天，海天相接处天地间。波涛滚滚江山美，海鸟翱翔日月闲。<|im_end|> <|im_start|>谢谢你，这篇文章很有用。不客气，我很高兴能够为您提供帮助。如果您还有其他问题或需求，随时可以对我说。<|im_end|> <|im_start|>你好，我想下载一个视频编辑软件，你有什么推荐吗？您好！当然，有很多选择。您想要免费

'<|im_start|>鉴别一组中文文章的风格和特点，例如官方、口语、文言等。需要提供样例文章才能准确鉴别不同的风格和特点。<|im_end|> <|im_start|>好的，现在帮我查一下今天的天气怎么样?今天的天气依据地区而异。请问你需要我帮你查询哪个地区的天气呢？<|im_end|> <|im_start|>打开闹钟功能，定一个明天早上七点的闹钟。好的，我已经帮您打开闹钟功能，闹钟将在明天早上七点准时响起。<|im_end|> <|im_start|>为以下场景写一句话描述：一个孤独的老人坐在公园长椅上看着远处。一位孤独的老人坐在公园长椅上凝视远方。<|im_end|> <|im_start|>非常感谢你的回答。请告诉我，这些数据是关于什么主题的？这些数据是关于不同年龄段的男女人口比例分布的。<|im_end|> <|im_start|>帮我想一个有趣的标题。这个挺有趣的："如何成为一名成功的魔术师" 调皮的标题往往会吸引读者的注意力。<|im_end|> <|im_start|>回答一个问题，地球的半径是多少？地球的平均半径约为6371公里，这是地球自赤道到两极的距离的平均值。<|im_end|> <|im_start|>识别文本中的语气，并将其分类为喜悦、悲伤、惊异等.文本：“今天是我的生日！”这个文本的语气是喜悦。<|im_end|><|im_start|>鉴别一组中文文章的风格和特点，例如官方、口语、文言等。需要提供样例文章才能准确鉴别不同的风格和特点。<|im_end|> <|im_start|>好的，现在帮我查一下今天的天气怎么样?今天的天气依据地区而异。请问你需要我帮你查询哪个地区的天气呢？<|im_end|> <|im_start|>打开闹钟功能，定一个明天早上七点的闹钟。好的，我已经帮您打开闹钟功能，闹钟将在明天早上七点准时响起。<|im_end|> <|im_start|>为以下场景写一句话描述：一个孤独的老人坐在公园长椅上看着远处。一位孤独的老人坐在公园长椅上凝视远方。<|im_end|> <|im_start|>非常感谢你的回答。请告诉我，这些数据是关于什么主题的？这些数据是关于不同年龄段的男女人口比例分布的。<|im_end|> <|im_start|>帮我想一个有趣的标题。这个挺有趣的："如何成为一名成功的魔术师" 调皮的标题往往

# 存在的问题
1.为什么outputs2  直接吐出的就是原句子

2.为什么outputs4  吐出来的new_tokens list竟然是空的，也就是没有任何词语吐出来
